# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f49107a1df0>
├── 'a' --> tensor([[ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080]])
└── 'x' --> <FastTreeValue 0x7f49107a1f40>
    └── 'c' --> tensor([[-0.8907,  1.6398, -1.4632, -1.8240],
                        [-0.5300,  2.2241,  0.4744, -0.4452],
                        [-1.1241,  1.6356,  1.2287, -0.6480]])

In [4]:
t.a

tensor([[ 1.2427, -0.8492, -0.0696],
        [ 0.6282,  0.9770,  1.4080]])

In [5]:
%timeit t.a

59.4 ns ± 0.0565 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f49107a1df0>
├── 'a' --> tensor([[ 0.0484, -0.3310, -0.2410],
│                   [-0.0704, -0.5577, -0.1897]])
└── 'x' --> <FastTreeValue 0x7f49107a1f40>
    └── 'c' --> tensor([[-0.8907,  1.6398, -1.4632, -1.8240],
                        [-0.5300,  2.2241,  0.4744, -0.4452],
                        [-1.1241,  1.6356,  1.2287, -0.6480]])

In [7]:
%timeit t.a = new_value

59.2 ns ± 0.0482 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080]]),
    x: Batch(
           c: tensor([[-0.8907,  1.6398, -1.4632, -1.8240],
                      [-0.5300,  2.2241,  0.4744, -0.4452],
                      [-1.1241,  1.6356,  1.2287, -0.6480]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2427, -0.8492, -0.0696],
        [ 0.6282,  0.9770,  1.4080]])

In [11]:
%timeit b.a

53.1 ns ± 0.0633 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.5370, -0.4997,  0.4948],
               [-0.8042,  1.1225, -0.9044]]),
    x: Batch(
           c: tensor([[-0.8907,  1.6398, -1.4632, -1.8240],
                      [-0.5300,  2.2241,  0.4744, -0.4452],
                      [-1.1241,  1.6356,  1.2287, -0.6480]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 0.166 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 31.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 297 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4859acfd60>
├── 'a' --> tensor([[[ 1.2427, -0.8492, -0.0696],
│                    [ 0.6282,  0.9770,  1.4080]],
│           
│                   [[ 1.2427, -0.8492, -0.0696],
│                    [ 0.6282,  0.9770,  1.4080]],
│           
│                   [[ 1.2427, -0.8492, -0.0696],
│                    [ 0.6282,  0.9770,  1.4080]],
│           
│                   [[ 1.2427, -0.8492, -0.0696],
│                    [ 0.6282,  0.9770,  1.4080]],
│           
│                   [[ 1.2427, -0.8492, -0.0696],
│                    [ 0.6282,  0.9770,  1.4080]],
│           
│                   [[ 1.2427, -0.8492, -0.0696],
│                    [ 0.6282,  0.9770,  1.4080]],
│           
│                   [[ 1.2427, -0.8492, -0.0696],
│                    [ 0.6282,  0.9770,  1.4080]],
│           
│                   [[ 1.2427, -0.8492, -0.0696],
│                    [ 0.6282,  0.9770,  1.4080]]])
└── 'x' --> <FastTreeValue 0x7f4859acff10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 45.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f486b0a42b0>
├── 'a' --> tensor([[ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080],
│                   [ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080],
│                   [ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080],
│                   [ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080],
│                   [ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080],
│                   [ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080],
│                   [ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080],
│                   [ 1.2427, -0.8492, -0.0696],
│                   [ 0.6282,  0.9770,  1.4080]])
└── 'x' --> <FastTreeValue 0x7f486b0a4040>
    └── 'c' --> tensor([[-0.8907,  1.6398, -1.4632, -1.8240],
                        [-0.5300,  2.2241,  0.4744, -0.4452],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 67.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.7 µs ± 73 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2427, -0.8492, -0.0696],
                [ 0.6282,  0.9770,  1.4080]],
       
               [[ 1.2427, -0.8492, -0.0696],
                [ 0.6282,  0.9770,  1.4080]],
       
               [[ 1.2427, -0.8492, -0.0696],
                [ 0.6282,  0.9770,  1.4080]],
       
               [[ 1.2427, -0.8492, -0.0696],
                [ 0.6282,  0.9770,  1.4080]],
       
               [[ 1.2427, -0.8492, -0.0696],
                [ 0.6282,  0.9770,  1.4080]],
       
               [[ 1.2427, -0.8492, -0.0696],
                [ 0.6282,  0.9770,  1.4080]],
       
               [[ 1.2427, -0.8492, -0.0696],
                [ 0.6282,  0.9770,  1.4080]],
       
               [[ 1.2427, -0.8492, -0.0696],
                [ 0.6282,  0.9770,  1.4080]]]),
    x: Batch(
           c: tensor([[[-0.8907,  1.6398, -1.4632, -1.8240],
                       [-0.5300,  2.2241,  0.4744, -0.4452],
                       [-1.1241,  1.6356,  1.2287, -0.6480]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080],
               [ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080],
               [ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080],
               [ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080],
               [ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080],
               [ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080],
               [ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080],
               [ 1.2427, -0.8492, -0.0696],
               [ 0.6282,  0.9770,  1.4080]]),
    x: Batch(
           c: tensor([[-0.8907,  1.6398, -1.4632, -1.8240],
                      [-0.5300,  2.2241,  0.4744, -0.4452],
                      [-1.1241,  1.6356,  1.2287, -0.6480],
                      [-0.8907,  1.6398, -1.4632, -1.8240],
                      [-0.5300,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 645 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 598 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
